# 決賽步驟
會先有一個預備的模型正在部屬中
1. 訓練 + 上傳到 s3
2. 部屬
3. 人工標記資料 + 拉到自己 notebook 訓練

## 0. 預備套件

In [12]:
!pip uninstall efficientnet_pytorch==0.7.1 -y
!pip uninstall librosa==0.6.0 -y
!pip uninstall numba==0.48.0 --user

Found existing installation: numba 0.52.0
Uninstalling numba-0.52.0:
  Successfully uninstalled numba-0.52.0


In [2]:
!pip install efficientnet_pytorch==0.7.1
!pip install librosa==0.6.0
!pip install numba==0.48.0 --user

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=52f62fd696925b18fd7560d06110329b0f5e07d9735a491c6a485cac7266f8db
  Stored in directory: /home/ec2-user/.cache/pip/wheels/63/17/7e/07f1c55a623c96ba9b291c5d2e4901afffb953b032c2470ca6
Successfully built efficientnet-pytorch
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 1.5 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 22.6 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 23.8 MB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.6.0-py3-none-any.whl size=1553494 sha256=c37fd0d82fdb7fd84a472a0f755cbd7e28302efefe3ae510a7c5404d0f851bbc
  Stored in directory: /home/ec2-user/.cache/pip/wheels/72/c0/f2/b00db6242414bafcbbca59c495a52a1bfd0574f29a7e498dc9
  Created wheel for audioread: filename=audioread-2.1.9-py3

In [1]:
# Base Package
import os
import time
import copy
import datetime
from pprint import pformat
import logging
import numpy as np
import pandas as pd 
from tqdm import tqdm_notebook
from tqdm import tqdm
import tqdm
import json
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

# Pytorch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.autograd import Variable
from torch.nn.modules.loss import _WeightedLoss
from torch.utils.data import Dataset, DataLoader
from efficientnet_pytorch import EfficientNet

# Our Package
from dataset import SoundDataset, get_melspec
from models import ModelEMA 
from metrics import roc_auc 
from ops import AdamW



from argparse import ArgumentParser
from pprint import pformat
import logging


import librosa 
from torch.utils.data import DataLoader
from metrics import cfm, classification_report, roc_auc
import flask

## 存佑駿爬的音檔
- 一個音檔就存成一個 npy
- 依照資料夾分類
- 根據搜索相關性可能只會用前 20 個搜尋結果
- 把表現不好的音檔刪掉，之後就不要再訓練了

In [71]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
devtmpfs        31426652       72  31426580   1% /dev
tmpfs           31437352        8  31437344   1% /dev/shm
/dev/xvda1     108305356 89592444  18612664  83% /
/dev/xvdf      515535104 53855112 439622956  11% /home/ec2-user/SageMaker


In [19]:
if not os.path.isdir(f'other_data'): os.mkdir(f'other_data')

In [72]:
os.path.isdir(f'../other_data')

True

In [ ]:
import pickle
with open('../filename.pickle', 'rb') as handle:
    other_data_path = pickle.load(handle)
for cat in other_data_path.keys():
    if not os.path.isdir(f'../other_data/{cat}'): os.mkdir(f'../other_data/{cat}')
    for url in other_data_path[cat]:
        file_name = url.split('/')[-1].split('.')[0]
        !wget -O temp.wav {url}
        mel_spec = get_melspec('./','temp')
        np.save(f'../other_data/{cat}/{file_name}.npy',mel_spec)

--2021-07-14 16:15:59--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-000.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.164.34
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.164.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21347064 (20M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]  20.36M  17.7MB/s    in 1.2s    

2021-07-14 16:16:00 (17.7 MB/s) - ‘temp.wav’ saved [21347064/21347064]

--2021-07-14 16:16:06--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-001.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.196.226
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.196.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10791872 (10M) [audio/wav

--2021-07-14 16:17:05--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-014.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.200.74
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.200.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3556744 (3.4M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   3.39M  5.95MB/s    in 0.6s    

2021-07-14 16:17:06 (5.95 MB/s) - ‘temp.wav’ saved [3556744/3556744]

--2021-07-14 16:17:07--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-015.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.17.222
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.17.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13602856 (13M) [audio/wav]
Sa

--2021-07-14 16:17:49--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-028.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.21.142
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.21.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7185792 (6.9M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   6.85M  9.60MB/s    in 0.7s    

2021-07-14 16:17:50 (9.60 MB/s) - ‘temp.wav’ saved [7185792/7185792]

--2021-07-14 16:17:52--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-029.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.203.2
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.203.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13178496 (13M) [audio/wav]
Savi

--2021-07-14 16:18:39--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-042.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.169.122
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.169.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8403712 (8.0M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   8.01M  9.94MB/s    in 0.8s    

2021-07-14 16:18:41 (9.94 MB/s) - ‘temp.wav’ saved [8403712/8403712]

--2021-07-14 16:18:43--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-043.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.187.94
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.187.94|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7869384 (7.5M) [audio/wav]


--2021-07-14 16:19:38--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-056.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.93.198
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.93.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4773152 (4.6M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   4.55M  8.00MB/s    in 0.6s    

2021-07-14 16:19:39 (8.00 MB/s) - ‘temp.wav’ saved [4773152/4773152]

--2021-07-14 16:19:40--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-057.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.93.198
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.93.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12105272 (12M) [audio/wav]
Sa

...

Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.184.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 882768 (862K) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>] 862.08K  2.13MB/s    in 0.4s    

2021-07-14 19:32:46 (2.13 MB/s) - ‘temp.wav’ saved [882768/882768]

--2021-07-14 19:32:47--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_06/s01-418.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.172.234
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.172.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 833376 (814K) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>] 813.84K  2.21MB/s    in 0.4s    

2021-07-14 19:32:47 (2.21 MB/s) - ‘temp.wav’ saved [833376/833376]

./ temp
--2021-07-14 19:32:48--  h

Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.67.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1066224 (1.0M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   1.02M  2.85MB/s    in 0.4s    

2021-07-14 19:33:12 (2.85 MB/s) - ‘temp.wav’ saved [1066224/1066224]

--2021-07-14 19:33:12--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_06/s01-432.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.86.62
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.86.62|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 907412 (886K) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>] 886.14K  2.16MB/s    in 0.4s    

2021-07-14 19:33:13 (2.16 MB/s) - ‘temp.wav’ saved [907412/907412]

--2021-07-14 19:33:13--  https://crawl

Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.107.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1785936 (1.7M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   1.70M  3.44MB/s    in 0.5s    

2021-07-14 19:33:33 (3.44 MB/s) - ‘temp.wav’ saved [1785936/1785936]

--2021-07-14 19:33:33--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_06/s01-446.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.107.0
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.107.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2054064 (2.0M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   1.96M  4.46MB/s    in 0.4s    

2021-07-14 19:33:34 (4.46 MB/s) - ‘temp.wav’ saved [2054064/2054064]

--2021-07-14 19:33:35--  https://c

Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.168.250|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17005728 (16M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]  16.22M  16.0MB/s    in 1.0s    

2021-07-14 19:34:07 (16.0 MB/s) - ‘temp.wav’ saved [17005728/17005728]

--2021-07-14 19:34:12--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_06/s01-460.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.216.146.238
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.216.146.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6252384 (6.0M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   5.96M  9.07MB/s    in 0.7s    

2021-07-14 19:34:13 (9.07 MB/s) - ‘temp.wav’ saved [6252384/6252384]

--2021-07-14 19:34:15--  h

Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.133.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 539376 (527K) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>] 526.73K  1.79MB/s    in 0.3s    

2021-07-14 19:34:52 (1.79 MB/s) - ‘temp.wav’ saved [539376/539376]

./ temp
--2021-07-14 19:34:52--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_06/s01-474.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.133.154
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.133.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3046608 (2.9M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   2.91M  5.89MB/s    in 0.5s    

2021-07-14 19:34:53 (5.89 MB/s) - ‘temp.wav’ saved [3046608/3046608]

--2021-07-14 19:34:54--

In [3]:
# if not os.path.exists('params.save_root'):
#     os.mkdir(params.save_root)
#     print("create folder: {}".format(params.save_root))
#     if not os.path.exists(os.path.join(params.save_root, 'snapshots')):
#         os.mkdir(os.path.join(params.save_root, 'snapshots'))
#     if not os.path.exists(os.path.join(params.save_root, 'log')):
#         os.mkdir(os.path.join(params.save_root, 'log'))

####################################
# data preparing & model training  #
####################################

# 訓練資料集
# train_sepc = []
# for file_name in tqdm.notebook.tqdm(meta_train.Filename):
#     train_sepc.append(get_melspec('train_data',file_name))
# train_sepc   

In [4]:
train_sepc = []
for file_name in tqdm.notebook.tqdm(meta_train.Filename):
    train_sepc.append(get_melspec('Final_Training_Dataset/train/',file_name))
train_sepc  

  0%|          | 0/1600 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: '/home/ec2-user/SageMaker/incremental_training_tbrain_dog/01-byoc/code/Final_Training_Dataset/train/train_00001.wav'

In [19]:
!unzip /Final_Training_Dataset/train.zip

unzip:  cannot find or open /Final_Training_Dataset/train.zip, /Final_Training_Dataset/train.zip.zip or /Final_Training_Dataset/train.zip.ZIP.


In [ ]:
def prepare_retraia

In [ ]:
def train()

In [87]:
%time

# 讀取資料
mel_spec = get_melspec('../Final_Training_Dataset/train/', 'train_00001')
mel_concat = np.stack((mel_spec, mel_spec, mel_spec)) / 255

# 轉格式
img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
wav = torch.tensor(img_test) 

# 模型輸入
model_input = wav.to(device, dtype=torch.float)
outputs = model(model_input)
outputs = torch.nn.functional.softmax(outputs, dim=1)
_, preds = torch.max(outputs, 1)

# Request
pred_label = preds.cpu().detach().numpy()
outputs = outputs.cpu().detach().numpy()

print(pred_label, outputs)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.3 µs
[9] [[0.10826567 0.12306647 0.141265   0.10384172 0.07344227 0.12768829
  0.08311921 0.04824285 0.04252069 0.14854777]]


In [3]:
train_sepc.shape

(1600, 128, 157)

In [12]:
# test_set = DogDataset(np.array(train_sepc))
test_set = SoundDataset(np.array(train_sepc),  meta_train.Label.values)
test_loader = DataLoader(test_set, batch_size=32, shuffle=True, num_workers=0, pin_memory=True)
for batch in tqdm_notebook(test_loader):
    imgs, labels = batch
images = imgs.to(device, dtype=torch.float)
labels = labels.to(device)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/50 [00:00<?, ?it/s]

In [110]:
!wget https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-001.wav

--2021-07-14 12:17:16--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_00/s01-001.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.172.194
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.172.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10791872 (10M) [audio/wav]
Saving to: ‘s01-001.wav’

s01-001.wav         100%[===================>]  10.29M  10.6MB/s    in 1.0s    

2021-07-14 12:17:18 (10.6 MB/s) - ‘s01-001.wav’ saved [10791872/10791872]



In [5]:
s = time.time()
model.eval()
mel_spec = get_melspec('../Final_Training_Dataset/train/', 'train_00042')
mel_concat = np.stack((mel_spec, mel_spec, mel_spec)) / 255

# 轉格式
img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
wav = torch.tensor(img_test) 
for model in model_list:
    # 讀取資料


    # 模型輸入
    model_input = wav.to(device, dtype=torch.float)
    outputs = model(model_input)
    outputs = torch.nn.functional.softmax(outputs, dim=1)
    _, preds = torch.max(outputs, 1)

    # Request
    pred_label = preds.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()

    print(pred_label, outputs)
print(time.time()-s)

NameError: name 'model' is not defined

In [26]:
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10,in_channels=3).to(device)
model.load_state_dict(torch.load('./model/eff_class10_0711/model_auc_0.9999218750000001.pth'))
model.eval()


Loaded pretrained weights for efficientnet-b0


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=

In [12]:
ls ../model2/eff_class10_0714_3

model_acc_fold_0.ckpt  model_auc_fold_0.ckpt  model_loss_fold_0.ckpt
model_acc_fold_1.ckpt  model_auc_fold_1.ckpt  model_loss_fold_1.ckpt
model_acc_fold_2.ckpt  model_auc_fold_2.ckpt  model_loss_fold_2.ckpt
model_acc_fold_3.ckpt  model_auc_fold_3.ckpt  model_loss_fold_3.ckpt
model_acc_fold_4.ckpt  model_auc_fold_4.ckpt  model_loss_fold_4.ckpt


## 2 訓練模型

In [ ]:
# 人工標記資料
import pickle
with open('../filename.pickle', 'rb') as handle:
    other_data_path = pickle.load(handle)
for cat in other_data_path.keys():
    if not os.path.isdir(f'../other_data/{cat}'): os.mkdir(f'../other_data/{cat}')
    for url in other_data_path[cat]:
        file_name = url.split('/')[-1].split('.')[0]
        !wget -O temp.wav {url}
        mel_spec = get_melspec('./','temp')
        np.save(f'../other_data/{cat}/{file_name}.npy',mel_spec)

In [18]:
new_meta_data_url = 'https://sagemaker-us-west-2-355444812467.s3.us-west-2.amazonaws.com/tomofun/1626487526.0505817/competition/meta_train.csv'

In [26]:
# def new_get_melspec(prefix, file_name):
#     y, sr = librosa.load(f'{prefix}/{file_name}.wav', sr = 8000)
#     if len(y)<sr*5:
#         print(prefix,file_name)
#         l = sr*5-len(y)
#         y = np.hstack([y,np.zeros(l)])
#     mel_spec = librosa.power_to_db(librosa.feature.melspectrogram(y, sr=sr, n_mels=128, n_fft=1024, hop_length=256))
#     return mel_spec

In [ ]:
# import datetime
from datetime import datetime, timedelta, timezone
datetime.now(timezone(timedelta(hours=8)))
meta_train = pd.read_csv('../Final_Training_Dataset/meta_train.csv')
meta_train
# 儲存資料(npy格式)
# np.save('./train_spec_class10_0711.npy', np.array(train_sepc))
# 讀取資料(npy格式)
train_sepc = np.load('../train_spec_class10_0711.npy')
# train_sepc

In [37]:
1+1

2

In [32]:
new_meta_data_name = '../s3-new-data2/competition/meta_train.csv'
new_meta_train = pd.read_csv(new_meta_data_name)
# 新的訓練資料集
new_train_sepc = []
for file_name in tqdm.notebook.tqdm(new_meta_train.Filename):
    try:
        new_train_sepc.append(get_melspec('../s3-new-data2/competition/train',file_name))
    except:
        pass
new_train_sepc = np.array(new_train_sepc)

  0%|          | 0/102 [00:00<?, ?it/s]

In [27]:
new_meta_data_name = '../s3-new-data/competition/meta_train.csv'
new_meta_train = pd.read_csv(new_meta_data_name)
# 新的訓練資料集
new_train_sepc = []
for file_name in tqdm.notebook.tqdm(new_meta_train.Filename):
    try:
        new_train_sepc.append(get_melspec('../s3-new-data/competition/train',file_name))
    except:
        pass
new_train_sepc = np.array(new_train_sepc)

  0%|          | 0/131 [00:00<?, ?it/s]

In [42]:
new_meta_data_namez = '../s3-new-data/competition/meta_train.csv'
new_meta_trainz = pd.read_csv(new_meta_data_namez)
# 新的訓練資料集
new_train_sepcz = []
for file_name in tqdm.notebook.tqdm(new_meta_trainz.Filename):
    try:
        new_train_sepcz.append(get_melspec('../s3-new-data/competition/train',file_name))
    except:
        pass
new_train_sepcz = np.array(new_train_sepcz)

  0%|          | 0/131 [00:00<?, ?it/s]

In [40]:
new_meta_data_name = '../s3-new-data3/competition/meta_train.csv'
new_meta_train = pd.read_csv(new_meta_data_name)
# 新的訓練資料集
new_train_sepc = []
for file_name in tqdm.notebook.tqdm(new_meta_train.Filename):
    new_train_sepc.append(get_melspec('../s3-new-data3/competition/train',file_name))
new_train_sepc = np.array(new_train_sepc)

  0%|          | 0/248 [00:00<?, ?it/s]

In [39]:
new_train_sepc.shape

(247, 128, 157)

In [44]:
train_sepc.shape

(2081, 128, 157)

In [43]:
# 這邊做 retrain 整合
meta_train = pd.concat([meta_train,new_meta_train])
train_sepc = np.r_[train_sepc,new_train_sepc]

In [45]:
# 5-fold 
# 參數設定
device = "cuda" if torch.cuda.is_available() else "cpu"

model_dir = 'model_efn'
save_model_dir = 'eff_class10_0717_0'
if not os.path.isdir(f'../{model_dir}/'):
    os.mkdir(f'../{model_dir}/')
if not os.path.isdir(f'../{model_dir}/{save_model_dir}'):
    os.mkdir(f'../{model_dir}/{save_model_dir}')

train_record = {}
# for i in ['accuracy', 'loss', 'auc']:
for i in ['auc']:
    for j in range(5):
        train_record[f'valid_{i}_{j}'] = []

        
# 是否要用外部資料
use_ema = True
# use_other_data = False
        
criterion_2 = nn.CrossEntropyLoss(reduce=False)
loss_thres = 1000

MAX_EPOCHS = 100000
patience_total = 5
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=321)
for fold, (train_index, val_index) in enumerate(skf.split(train_sepc, meta_train.Label.values)):
    
#     if fold > 0:
#         break
    print(f'---- fold{fold} --------------------------------------')

    print("Preparing training/validation data...")
    trn_dataset = SoundDataset(np.array(train_sepc)[train_index], meta_train.Label.values[train_index])
    val_dataset = SoundDataset(np.array(train_sepc)[val_index], meta_train.Label.values[val_index], mode='val')
    # num_workers：使用多進程加載的進程數、pin_memory：是否將數據保存在pin memory區，pin memory中的數據轉到GPU會快一些
    trn_loader = DataLoader(trn_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)

    print("Preparing efficientnet model...")
    model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10,in_channels=3).to(device)
#     model.load_state_dict(torch.load('./model/eff_class10_0711/model_auc_fold_0.ckpt'))
    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=0.001, weight_decay=0.01)  
    
    print("Preparing model path...")
    best_loss = 10; best_acc = 0.0; patience = 0; best_auc = 0.5    
#     model_path = f'./model/{save_model_dir}/model_loss_fold_{fold}.ckpt'
#     model_path_2 = f'./model/{save_model_dir}/model_acc_fold_{fold}.ckpt'
    model_path_3 = f'../{model_dir}/{save_model_dir}/model_auc_fold_{fold}.ckpt'

    if use_ema:
        ema_model = ModelEMA(model, decay=0.99)
        
    print("Starting train model...")
    # ---- train -------------------------------------------------
    for epoch in range(0,MAX_EPOCHS):
#         loss_thres = (1-best_auc)**(1/2)
        loss_thres = 0.7
        print(f"epoch: {epoch}...threshold: {loss_thres}")
        # ---------- Training ----------
        model.train()
        train_loss = 0; reg_loss = 0; correct = 0; total = 0    
        
        

        # Iterate the training set by batches.
        # for batch in tqdm.notebook.tqdm(trn_loader):
        for batch in tqdm.notebook.tqdm(trn_loader):

#             # ---------- mixup -----------
            
#             other_data_np_x = []
#             other_data_np_y = []
#             # root_other_dir = 
#             for i in range(10):
#                 root_other_dir = f'../other_data/category_0{i}/'
#                 all_npy = os.listdir(root_other_dir)
#                 if all_npy == []:
#                     continue
#                 for i in range(10):
#                     data_temp = np.load(root_other_dir+np.random.choice(all_npy))
#                     rs = np.random.choice(data_temp.shape[1]-157) if data_temp.shape[1] > 157 else 0
#                     data_temp = data_temp[:,rs:rs+157]
#                     other_data_np_x.append(np.stack((data_temp, data_temp, data_temp))/255)
#                     other_data_np_y.append(i)
#             od_x = torch.tensor(np.r_[other_data_np_x])
#             od_y = torch.tensor(np.r_[other_data_np_y])
            
#             inputs_others, targets_others = od_x.cuda(), od_y.cuda()
#             inputs_others = inputs_others.float()
#             outputs = model(inputs_others)
#             loss_other = criterion_2( outputs, targets_others)
#             loss_other = -nn.Threshold(-loss_thres,0)(-loss_other)
# #             print(loss_other)
#             loss_other = loss_other.mean() # loss > 0.3 的樣本直接把 loss 歸 0，接著平均。
            
            inputs, targets = batch

#             inputs = torch.cat([od_x,batch[0]],0)
#             targets = torch.cat([od_y,batch[1]],0)
            
            
            inputs, targets = inputs.cuda(), targets.cuda()
            inputs = inputs.float()
            outputs = model(inputs)
#             print(criterion_2( outputs, targets))

#             loss = criterion( outputs, targets) + 1 * loss_other
            loss = criterion(outputs, targets)
            
            
            
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if use_ema:
                ema_model.update(model)

        # ---------- Validation ----------
        if use_ema:
            test_model = ema_model.ema
        else:
            test_model = model
        test_model.eval()

        # Validation 紀錄
        valid_loss = []; valid_accs = []; val_label = []; val_pred = []; val_pred_p = []
        # Iterate the validation set by batches.
        for batch in val_loader:

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch

            # 驗證階段不用梯度的資訊，取消梯度以加速運算
            with torch.no_grad():
                logits = test_model(imgs.to(device).float())

            # 計算每批 loss 跟 accuracy
            loss = criterion(logits, labels.to(device))   
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean() 
            val_pred.append(logits.argmax(dim=-1).to('cpu').numpy())
            val_label.append(labels.numpy())
            val_pred_p.append(nn.Softmax(dim=-1)(logits[:,:10]).to('cpu').numpy())
            valid_loss.append(loss.item())
            valid_accs.append(acc.item())

        # 計算整體 loss 跟 accuracy
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        cm = pd.crosstab(np.hstack(val_label),np.hstack(val_pred))
        # print(cm)roc_auc_score(y_true, y_score, multi_class=multi_class, average=average)
        valid_auc = roc_auc(np.eye(10)[list(np.hstack(val_label))],np.vstack(val_pred_p))
        
        
#         # ==== 計算不同方法的 auc =====
#         for ave in ['micro','macro','weighted','samples']:
#             auc_temp = roc_auc(np.eye(10)[list(np.hstack(val_label))],np.vstack(val_pred_p),average=ave)
#             print(auc_temp)
        
#         train_record[f'valid_loss_{fold}'].append(valid_loss)
#         train_record[f'valid_accuracy_{fold}'].append(valid_acc)
        train_record[f'valid_auc_{fold}'].append(valid_auc)

        if valid_auc > best_auc:
            best_auc = valid_auc
            torch.save(test_model.state_dict(), model_path_3)
            print(f'epoch_{epoch} ','saving model with auc {:.5f}'.format(valid_auc))
            patience=0
        else:
            patience+=1
            if patience>patience_total:
                print('auc')
                break


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


---- fold0 --------------------------------------
Preparing training/validation data...
Preparing efficientnet model...
Loaded pretrained weights for efficientnet-b0
Preparing model path...
Starting train model...
epoch: 0...threshold: 0.7


  0%|          | 0/52 [00:00<?, ?it/s]

epoch_0  saving model with auc 0.55350
epoch: 1...threshold: 0.7


  0%|          | 0/52 [00:00<?, ?it/s]

...

epoch: 31...threshold: 0.7


  0%|          | 0/53 [00:00<?, ?it/s]

epoch: 32...threshold: 0.7


  0%|          | 0/53 [00:00<?, ?it/s]

auc


In [6]:
# import datetime
from datetime import datetime, timedelta, timezone
datetime.now(timezone(timedelta(hours=8)))

datetime.datetime(2021, 7, 17, 4, 43, 32, 596489, tzinfo=datetime.timezone(datetime.timedelta(0, 28800)))

In [42]:
# 加外部資料
for i in range(5):
    print(max(train_record[f'valid_auc_{i}']))

ValueError: max() arg is an empty sequence

In [7]:
# 加外部資料
for i in range(5):
    print(max(train_record[f'valid_auc_{i}']))

0.9921041666666666
0.9910148809523809
0.9973906250000001
0.9899642857142859
0.9937758570854583


In [20]:
# 沒加外部資料
for i in range(5):
    print(max(train_record[f'valid_auc_{i}']))

0.9928906249999999
0.9931785714285712
0.9956711309523809
0.9900751488095241
0.9945308739759346


In [21]:
train_record_0717 = train_record.copy()

In [24]:
train_record

{'valid_auc_0': [],
 'valid_auc_1': [],
 'valid_auc_2': [],
 'valid_auc_3': [],
 'valid_auc_4': []}

In [44]:
# !tar -zcvf model_0715_liyu.tar.gz eff_class10_0714_3

# !tar -zcvf '../{model_dir}/{save_model_dir}/model_0715_0.tar.gz' '../{model_dir}/{save_model_dir}/'

MODEL_PATH = 's3://sagemaker-us-west-2-355444812467/model'
model_data_s3_uri = f'{MODEL_PATH}/model_0715_0.tar.gz'
# !aws s3 cp model.tar.gz $model_data_s3_uri

tar: eff_class10_0714_3: Cannot stat: No such file or directory
tar: Exiting with failure status due to previous errors


In [46]:
# !tar -zcvf model_0715_liyu.tar.gz eff_class10_0714_3
MODEL_PATH = 's3://sagemaker-us-west-2-355444812467/model'
model_data_s3_uri = f'{MODEL_PATH}/model_0715_liyu.tar.gz'
# !aws s3 cp model.tar.gz $model_data_s3_uri
model_data_s3_uri

's3://sagemaker-us-west-2-355444812467/model/model_0715_liyu.tar.gz'

tar -zcvf model_0725_liyu.tar.gz eff_class10_0714_3

aws s3 cp model_0725_liyu.tar.gz s3://sagemaker-us-west-2-355444812467/model/model_0725_liyu.tar.gz

In [26]:
model_data_s3_uri = 's3://sagemaker-us-west-2-355444812467/model/model_test.tar.gz'


NameError: name 'MODEL_PATH' is not defined

In [62]:
'../train_data/train_00002.wav'

<function dataset.get_melspec(prefix, file_name)>

In [63]:
    mel_spec = get_melspec('../train_data/','train_00002')
    mel_concat = np.stack((mel_spec, mel_spec, mel_spec))/255

    # Reshape data to tensor
    img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
    wav = torch.tensor(img_test) 

In [64]:
    model_input = wav.to(device, dtype=torch.float)
    outputs_list = []
    for i in range(15):
        outputs = model_list[i](model_input)
        outputs_list.append(torch.nn.functional.softmax(outputs, dim=1))
    outputs = torch.stack(outputs_list, dim=0).sum(0)/15
    _, preds = torch.max(outputs, 1)

In [65]:
outputs

tensor([[0.5787, 0.0186, 0.1696, 0.0311, 0.0136, 0.0054, 0.0250, 0.0695, 0.0049,
         0.0837]], device='cuda:0', grad_fn=<DivBackward0>)

In [59]:
    # # model input format
    # model_input = wav.to(device, dtype=torch.float)
    # outputs = model(model_input)
    # outputs = torch.nn.functional.softmax(outputs, dim=1)
    # _, preds = torch.max(outputs, 1)

tensor([[0.2423, 0.0065, 0.0341, 0.1420, 0.0366, 0.1491, 0.0365, 0.1461, 0.0190,
         0.1877]], device='cuda:0', grad_fn=<DivBackward0>)

In [58]:
_

tensor([0.2423], device='cuda:0', grad_fn=<MaxBackward0>)

In [ ]:
1+1

In [57]:
preds

tensor([0], device='cuda:0')

In [55]:
torch.stack(outputs_list, dim=0).sum(0)/15

tensor([[0.2423, 0.0065, 0.0341, 0.1420, 0.0366, 0.1491, 0.0365, 0.1461, 0.0190,
         0.1877]], device='cuda:0', grad_fn=<DivBackward0>)

In [52]:
outputs_list

[tensor([[0.1818, 0.0059, 0.0357, 0.0476, 0.0576, 0.0808, 0.0519, 0.1721, 0.0181,
          0.3484]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.8746, 0.0012, 0.0088, 0.0034, 0.0020, 0.0109, 0.0026, 0.0106, 0.0016,
          0.0844]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.0919, 0.0114, 0.0686, 0.2240, 0.0218, 0.0704, 0.0906, 0.2329, 0.0461,
          0.1422]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.0227, 0.0017, 0.0249, 0.3285, 0.0109, 0.2818, 0.0210, 0.1396, 0.0044,
          0.1646]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.0424, 0.0121, 0.0369, 0.1013, 0.0875, 0.2785, 0.0176, 0.1750, 0.0245,
          0.2243]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.1763, 0.0063, 0.0225, 0.0637, 0.0671, 0.1504, 0.0491, 0.1730, 0.0198,
          0.2718]], device='cuda:0', grad_fn=<SoftmaxBackward>),
 tensor([[0.8746, 0.0012, 0.0088, 0.0034, 0.0020, 0.0109, 0.0026, 0.0106, 0.0016,
          0.0844]], device='cuda:0',

In [ ]:
    outputs = model(model_input)
    outputs = torch.nn.functional.softmax(outputs, dim=1)
    _, preds = torch.max(outputs, 1)

In [37]:
torchvisionimgs[0].

torch.Size([3, 128, 157])

In [38]:
mel_spec = get_melspec('../Final_Training_Dataset/train/', 'train_00042')
mel_concat = np.stack((mel_spec, mel_spec, mel_spec)) / 255

# 轉格式
img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
wav = torch.tensor(img_test) 

In [44]:
model_input = wav.to(device, dtype=torch.float)

In [39]:
    mel_spec = get_melspec('../Final_Training_Dataset/train/', 'train_00042')
    mel_concat = np.stack((mel_spec, mel_spec, mel_spec)) / 255

    # 轉格式
    img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
    wav = torch.tensor(img_test) 

    # 模型輸入
    model_input = wav.to(device, dtype=torch.float)
    outputs = model(model_input)
    outputs = torch.nn.functional.softmax(outputs, dim=1)
    _, preds = torch.max(outputs, 1)

    # Request
    pred_label = preds.cpu().detach().numpy()
    outputs = outputs.cpu().detach().numpy()


tensor([[[[-0.1355, -0.1318, -0.1353,  ..., -0.1271, -0.1332, -0.1380],
          [-0.1475, -0.1703, -0.1947,  ..., -0.1365, -0.1947, -0.1810],
          [-0.1680, -0.1875, -0.1947,  ..., -0.1335, -0.1947, -0.1806],
          ...,
          [-0.1775, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947],
          [-0.1869, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947],
          [-0.1923, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947]],

         [[-0.1355, -0.1318, -0.1353,  ..., -0.1271, -0.1332, -0.1380],
          [-0.1475, -0.1703, -0.1947,  ..., -0.1365, -0.1947, -0.1810],
          [-0.1680, -0.1875, -0.1947,  ..., -0.1335, -0.1947, -0.1806],
          ...,
          [-0.1775, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947],
          [-0.1869, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947],
          [-0.1923, -0.1947, -0.1947,  ..., -0.1947, -0.1947, -0.1947]],

         [[-0.1355, -0.1318, -0.1353,  ..., -0.1271, -0.1332, -0.1380],
          [-0.1475, -0.1703, -

In [43]:
wav.shape

torch.Size([1, 3, 128, 157])

In [45]:
model_list[5](model_input)

tensor([[ 1.1587, -2.1793, -0.9017,  0.1409,  0.1928,  0.9995, -0.1189,  1.1399,
         -1.0293,  1.5915]], device='cuda:0', grad_fn=<AddmmBackward>)

In [55]:
prefix = './'
# prefix = '/opt/ml/'
# prefix = '/opt/ml/'
model_list = []
# for metric in ['loss','auc','acc']:
for metric in ['auc']:
    for fold in [0,1,2,3,4]:
        model_name = f'../model_efn/eff_class10_0714_3/model_{metric}_fold_{fold}.ckpt'
#         model_path = os.path.join(prefix, 'model', model_name)  
        model_path = os.path.join( model_name)  
        model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10,in_channels=3).to(device)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        model = model.to(device)
        mel_spec = get_melspec('../Final_Training_Dataset/train/', 'train_00090')
        mel_concat = np.stack((mel_spec, mel_spec, mel_spec)) / 255

        # 轉格式
        img_test = np.reshape(mel_concat, (1, mel_concat.shape[0], mel_concat.shape[1], mel_concat.shape[2]))
        wav = torch.tensor(img_test) 

        # 模型輸入
        model_input = wav.to(device, dtype=torch.float)
        outputs = model(model_input)
        outputs = torch.nn.functional.softmax(outputs, dim=1)
        _, preds = torch.max(outputs, 1)

        # Request
        pred_label = preds.cpu().detach().numpy()
        outputs = outputs.cpu().detach().numpy()
        print(outputs)
#         model_list.append(model)
        

Loaded pretrained weights for efficientnet-b0
[[9.9969661e-01 9.7874672e-06 2.0311782e-04 1.6352585e-06 1.3172881e-06
  1.1388759e-06 7.4199043e-06 4.3319087e-05 2.9797004e-05 5.8401070e-06]]
Loaded pretrained weights for efficientnet-b0
[[9.9996471e-01 5.3794901e-07 4.1850803e-06 8.0189483e-07 2.2313937e-08
  6.1871090e-09 2.0412110e-05 9.1789298e-07 3.3357749e-06 5.0974431e-06]]
Loaded pretrained weights for efficientnet-b0
[[9.9991286e-01 1.1096951e-06 9.4199941e-06 8.4084485e-07 1.2863306e-07
  3.2854103e-08 3.4701000e-07 6.1778596e-06 5.1452691e-05 1.7542616e-05]]
Loaded pretrained weights for efficientnet-b0
[[9.9993396e-01 1.0207347e-06 2.3991064e-05 9.3526842e-07 8.1278579e-08
  3.5634240e-08 2.4223013e-05 3.8556377e-06 5.2030271e-07 1.1426993e-05]]
Loaded pretrained weights for efficientnet-b0
[[9.9998283e-01 4.2678924e-08 3.6484585e-06 5.7370717e-06 8.5482853e-08
  3.0040578e-08 2.8323737e-07 4.7520308e-07 2.9556639e-07 6.5560298e-06]]


In [ ]:
tar zcvf FileName.tar.gz DirName

In [ ]:
!wget -O model.tar.gz $source_model_data_url

MODEL_PATH = f's3://{bucket}/model'
model_data_s3_uri = f'{MODEL_PATH}/model.tar.gz'
!aws s3 cp model.tar.gz $model_data_s3_uri

In [12]:
!df

Filesystem     1K-blocks      Used Available Use% Mounted on
devtmpfs        31426652        72  31426580   1% /dev
tmpfs           31437352         8  31437344   1% /dev/shm
/dev/xvda1     108305356 108288972         0 100% /
/dev/xvdf      515535104  35460280 458017788   8% /home/ec2-user/SageMaker


In [10]:
ls model/eff_class10_0714_3

model_acc_fold_0.ckpt  model_auc_fold_0.ckpt  model_loss_fold_0.ckpt
model_acc_fold_1.ckpt  model_auc_fold_1.ckpt  model_loss_fold_1.ckpt
model_acc_fold_2.ckpt  model_auc_fold_2.ckpt  model_loss_fold_2.ckpt
model_acc_fold_3.ckpt  model_auc_fold_3.ckpt  model_loss_fold_3.ckpt
model_acc_fold_4.ckpt  model_auc_fold_4.ckpt  model_loss_fold_4.ckpt


In [74]:
# arn:aws:sagemaker:us-west-2:355444812467:endpoint/audio-1626195929
                    
api_endpoint = "https://runtime.sagemaker.us-west-2.amazonaws.com/endpoints/audio-1626195929/invocations"

!curl -X POST -H 'content-type: application/octet-stream' -H 'x-api-key: Y2hvdW44MTEzNjU6ZXJpYzA5MTYwNjY1NzU' --data-binary @../train_data/train_00002.wav    $api_endpoint                    

{"message":"Missing Authentication Token"}

In [76]:
import requests
import json

voice_data = '../train_data/train_00003.wav'
api_endpoint = "https://runtime.sagemaker.us-west-2.amazonaws.com/endpoints/audio-1626195929/invocations"

headers = {
    "Authorization": "Y2hvdW44MTEzNjU6ZXJpYzA5MTYwNjY1NzU",
    "Content-Type": "application/octet-stream",
}

r = requests.post(api_endpoint,
                  headers=headers, 
                  data=open(voice_data,'rb').read())

In [47]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00001.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"errorMessage": "An error occurred (ValidationError) when calling the InvokeEndpoint operation: 1 validation error detected: Value at 'body' failed to satisfy constraint: Member must not be null", "errorType": "ValidationError", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 26, in lambda_handler\n    Body=payload)\n", "  File \"/var/runtime/botocore/client.py\", line 357, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/runtime/botocore/client.py\", line 676, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}

## 爬蟲資料

In [10]:
import pickle
# with open('filename.pickle', 'rb') as handle:
#     pickle.dump(folder, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../../../crawl-data/filename.pickle', 'rb') as f:
    crawl_data = pickle.load(f)

In [11]:
for i,cat in enumerate(crawl_data.keys()):
    print(cat,len(crawl_data[cat]),'/',len(os.listdir(f'../other_data/category_0{i}')))

category_00 566 / 319
category_01 148 / 146
category_02 216 / 216
category_03 57 / 0
category_04 754 / 331
category_05 1404 / 952
category_06 900 / 832
category_07 2206 / 1910
category_08 301 / 0
category_09 1289 / 246


In [12]:
os.listdir(f'../other_data/{cat}')

['s01-100.npy',
 's01-041.npy',
 's01-223.npy',
 's01-142.npy',
 's01-198.npy',
 's01-196.npy',
 's01-149.npy',
 's01-054.npy',
 's01-003.npy',
 's01-197.npy',
 's01-194.npy',
 's01-134.npy',
 's01-174.npy',
 's01-091.npy',
 's01-007.npy',
 's01-123.npy',
 's01-232.npy',
 's01-027.npy',
 's01-164.npy',
 's01-183.npy',
 's01-016.npy',
 's01-156.npy',
 's01-030.npy',
 's01-210.npy',
 's01-032.npy',
 's01-105.npy',
 's01-081.npy',
 's01-180.npy',
 's01-203.npy',
 's01-135.npy',
 's01-120.npy',
 's01-244.npy',
 's01-099.npy',
 's01-106.npy',
 's01-199.npy',
 's01-152.npy',
 's01-048.npy',
 's01-117.npy',
 's01-124.npy',
 's01-168.npy',
 's01-069.npy',
 's01-217.npy',
 's01-079.npy',
 's01-154.npy',
 's01-031.npy',
 's01-188.npy',
 's01-035.npy',
 's01-173.npy',
 's01-245.npy',
 's01-159.npy',
 's01-094.npy',
 's01-070.npy',
 's01-151.npy',
 's01-103.npy',
 's01-207.npy',
 's01-182.npy',
 's01-206.npy',
 's01-064.npy',
 's01-112.npy',
 's01-071.npy',
 's01-158.npy',
 's01-087.npy',
 's01-22

In [13]:
for cat in crawl_data.keys():
    print(cat,len(crawl_data[cat]))
    for url in crawl_data[cat]:
        file_name = url.split('/')[-1].split('.')[0]
        ft = url.split('/')[-1].split('.')[1]
        if file_name+'.npy' in os.listdir(f'../other_data/{cat}') or ft=='mp3':
            continue
        !wget -O temp.wav {url}
        mel_spec = get_melspec('./','temp')
        np.save(f'../other_data/{cat}/{file_name}.npy',mel_spec)

category_00 566
category_01 148
category_02 216
category_03 57
category_04 754
category_05 1404
category_06 900
category_07 2206
category_08 301
category_09 1289
--2021-07-16 18:36:21--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_09/s01-246.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.44.248
Connecting to crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)|52.217.44.248|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7370880 (7.0M) [audio/wav]
Saving to: ‘temp.wav’

temp.wav            100%[===================>]   7.03M  9.45MB/s    in 0.7s    

2021-07-16 18:36:23 (9.45 MB/s) - ‘temp.wav’ saved [7370880/7370880]

--2021-07-16 18:36:25--  https://crawl-voice.s3.us-east-1.amazonaws.com/category_09/s01-247.wav
Resolving crawl-voice.s3.us-east-1.amazonaws.com (crawl-voice.s3.us-east-1.amazonaws.com)... 52.217.162.178
Connecting to crawl-voice.s3.us-east-1.amazonaws

KeyboardInterrupt: 

In [ ]:
import pickle
with open('../filename.pickle', 'rb') as handle:
    other_data_path = pickle.load(handle)
for cat in other_data_path.keys():
    if not os.path.isdir(f'../other_data/{cat}'): os.mkdir(f'../other_data/{cat}')
    for url in other_data_path[cat]:
        file_name = url.split('/')[-1].split('.')[0]
        !wget -O temp.wav {url}
        mel_spec = get_melspec('./','temp')
        np.save(f'../other_data/{cat}/{file_name}.npy',mel_spec)

In [ ]:
for

##  人工標記資料處理

In [58]:
# !unzip ../private_test.zip
# !unzip ../public_test.zip

In [68]:
folder = '../private_test'
for file_name in os.listdir('../private_test'):
    !curl -X POST -H 'content-type: application/octet-stream' \
    -H 'x-api-key: 0B22878B03FE197EF8D6' \
    --data-binary @{folder}/{file_name} \
    'https://pdtujuf6u4.execute-api.us-west-2.amazonaws.com/dev/classify'
    time.sleep(2)

{"label": 2, "probability": [2.3058308215695433e-05, 0.0023909432347863913, 0.9975711703300476, 1.2819239145756e-06, 8.476836654836006e-08, 1.2231690789121785e-06, 6.566640422533965e-07, 2.786102868412854e-06, 3.1306223036153824e-07, 8.451956091448665e-06]}{"label": 5, "probability": [1.761039311531931e-05, 2.2254607756622136e-05, 1.2973617231182288e-05, 0.0003194464079570025, 6.141074118204415e-05, 0.99934321641922, 1.7133339497377165e-05, 7.646361336810514e-05, 5.003375918022357e-06, 0.00012447542394511402]}{"label": 6, "probability": [0.005957537330687046, 0.2625618875026703, 0.00021049227507319301, 0.0007048180559650064, 0.006923270877450705, 0.23931685090065002, 0.3999711871147156, 0.00018587693921290338, 0.010991932824254036, 0.07317619770765305]}{"label": 6, "probability": [0.008938753977417946, 0.4009783864021301, 0.00017300236504524946, 0.00020989449694752693, 0.003032408654689789, 0.0002965212916024029, 0.5220288634300232, 8.240908209700137e-05, 0.010959881357848644, 0.053299

KeyboardInterrupt: 

In [60]:
os.listdir('../public_test')

['public_02930.wav',
 'public_00952.wav',
 'public_04553.wav',
 'public_02724.wav',
 'public_02818.wav',
 'public_00920.wav',
 'public_02361.wav',
 'public_03282.wav',
 'public_05809.wav',
 'public_09005.wav',
 'public_04439.wav',
 'public_01071.wav',
 'public_05369.wav',
 'public_08014.wav',
 'public_05277.wav',
 'public_09031.wav',
 'public_02610.wav',
 'public_03838.wav',
 'public_01319.wav',
 'public_08566.wav',
 'public_00293.wav',
 'public_06613.wav',
 'public_07499.wav',
 'public_08904.wav',
 'public_00784.wav',
 'public_08769.wav',
 'public_08249.wav',
 'public_03878.wav',
 'public_08973.wav',
 'public_07848.wav',
 'public_05475.wav',
 'public_08207.wav',
 'public_05417.wav',
 'public_04223.wav',
 'public_01228.wav',
 'public_00745.wav',
 'public_07823.wav',
 'public_00568.wav',
 'public_08074.wav',
 'public_01102.wav',
 'public_05576.wav',
 'public_01242.wav',
 'public_03214.wav',
 'public_07507.wav',
 'public_04269.wav',
 'public_00495.wav',
 'public_09777.wav',
 'public_0960

In [ ]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @./Final_Training_Dataset/train/train_00001.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

In [62]:
!curl -X POST -H 'content-type: application/octet-stream' \
-H 'x-api-key: 0B22878B03FE197EF8D6' \
--data-binary @../public_test/public_02930.wav \
'https:///rbrdok3cva.execute-api.us-west-2.amazonaws.com/dev/classify'

{"errorMessage": "An error occurred (ValidationError) when calling the InvokeEndpoint operation: Endpoint audio-1626202830 of account 355444812467 not found.", "errorType": "ValidationError", "stackTrace": ["  File \"/var/task/lambda_function.py\", line 34, in lambda_handler\n    Body=payload)\n", "  File \"/var/runtime/botocore/client.py\", line 357, in _api_call\n    return self._make_api_call(operation_name, kwargs)\n", "  File \"/var/runtime/botocore/client.py\", line 676, in _make_api_call\n    raise error_class(parsed_response, operation_name)\n"]}

In [ ]:
public_02930